#### Game

In [10]:
import pygame
import sys

# Initialisierung
pygame.init()
CLOCK = pygame.time.Clock()
SCREEN = pygame.display.set_mode((1280, 720))
pygame.display.set_caption("YOLOv5 Game")

# Startposition des Spielers
X_POSITION, Y_POSITION = 50, 440

# Lade Schriftart
GAME_FONT = pygame.font.Font("assets/MotleyForcesRegular-w1rZ3.ttf", 24)

# Spielgeschwindigkeit
GAME_SPEED = 5

# Spieler-Klasse
class Player(pygame.sprite.Sprite):
    def __init__(self, x_pos, y_pos):
        super().__init__()

        # Animationen für das Laufen, Ducken und Springen
        self.running_sprites = [
            pygame.transform.scale(pygame.image.load("assets/Dino1.png"), (80, 100)),
            pygame.transform.scale(pygame.image.load("assets/Dino2.png"), (80, 100))
        ]
        self.ducking_sprites = [
            pygame.transform.scale(pygame.image.load("assets/DinoDucking1.png"), (110, 60)),
            pygame.transform.scale(pygame.image.load("assets/DinoDucking2.png"), (110, 60))
        ]
        self.jumping_surface = pygame.transform.scale(pygame.image.load("assets/DinoJumping.png"), (80, 100))

        self.x = x_pos
        self.y = y_pos

        self.current_image = 0
        self.image = self.running_sprites[self.current_image]
        self.rect = self.image.get_rect(center=(self.x, self.y))

        self.velocity = 15
        self.gravity = 1.5
        self.ducking = False
        self.is_jumping = False

    def update(self):
        self.animate()
        self.apply_gravity()
        self.move()

    def animate(self):
        self.current_image += 0.05
        if self.current_image >= 2:
            self.current_image = 0

        if self.ducking:
            self.image = self.ducking_sprites[int(self.current_image)]
        elif self.is_jumping:
            self.image = self.jumping_surface
        else:
            self.image = self.running_sprites[int(self.current_image)]

    def duck(self):
        self.ducking = True
        self.rect.centery = 460 

    def unduck(self):
        self.ducking = False
        self.rect.centery = 440

    def apply_gravity(self):
        if self.rect.centery <= 440:
            self.rect.centery += self.gravity

    def jump(self):
        if self.rect.centery >= 440:
            self.velocity = -30
            self.is_jumping = True

    def move(self):
        if self.is_jumping:
            self.rect.centery += self.velocity
            self.velocity += self.gravity
            if self.rect.centery >= 440:
                self.rect.centery = 440
                self.velocity = 0
                self.is_jumping = False

PAUSE_COLOR = (100, 100, 100, 200)
PAUSE_FONT_COLOR = (255, 255, 255)
paused = False  # Initialer Pause-Zustand

def draw_pause_menu():
    # Erstellen einer leicht transparenten grauen Schicht
    pause_surface = pygame.Surface(SCREEN.get_size(), pygame.SRCALPHA)
    pause_surface.fill((100, 100, 100, 128))  # Leicht transparentes Grau
    SCREEN.blit(pause_surface, (0, 0))

    # Zeichnen des Hintergrunds und des Spielers
    SCREEN.blit(ground, (ground_x, 0))
    SCREEN.blit(ground, (ground_x + 1280, 0))
    player_group.draw(SCREEN)

    pause_text = GAME_FONT.render("PAUSE", True, PAUSE_FONT_COLOR)
    pause_rect = pause_text.get_rect(center=(SCREEN.get_width() // 2, SCREEN.get_height() // 2 - 40))
    SCREEN.blit(pause_text, pause_rect)

    quit_text = GAME_FONT.render("QUIT", True, PAUSE_FONT_COLOR)
    quit_rect = quit_text.get_rect(center=(SCREEN.get_width() // 2, SCREEN.get_height() // 2 + 40))
    SCREEN.blit(quit_text, quit_rect)

    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.MOUSEBUTTONDOWN and quit_rect.collidepoint(event.pos):
            pygame.quit()
            sys.exit()





# Hintergrund und Spielergruppe
ground = pygame.image.load("assets/background.png")
ground = pygame.transform.scale(ground, (1280, 900))
ground_x = 0
ground_rect = ground.get_rect(center=(640, 400))

player_group = pygame.sprite.GroupSingle()
player = Player(X_POSITION, Y_POSITION)
player_group.add(player)





SystemExit: 

C:\Users\saniy\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
%tb

SystemExit: 

In [13]:
import pygame
import sys

# Initialisierung
pygame.init()
CLOCK = pygame.time.Clock()
SCREEN = pygame.display.set_mode((1280, 720))
pygame.display.set_caption("YOLOv5 Game")

# Startposition des Spielers
X_POSITION, Y_POSITION = 50, 440

# Lade Schriftart
GAME_FONT = pygame.font.Font("assets/MotleyForcesRegular-w1rZ3.ttf", 60)

# Spielgeschwindigkeit
GAME_SPEED = 5

# Spieler-Klasse
class Player(pygame.sprite.Sprite):
    def __init__(self, x_pos, y_pos):
        super().__init__()

        # Animationen für das Laufen, Ducken und Springen
        self.running_sprites = [
            pygame.transform.scale(pygame.image.load("assets/Dino1.png"), (80, 100)),
            pygame.transform.scale(pygame.image.load("assets/Dino2.png"), (80, 100))
        ]
        self.ducking_sprites = [
            pygame.transform.scale(pygame.image.load("assets/DinoDucking1.png"), (110, 60)),
            pygame.transform.scale(pygame.image.load("assets/DinoDucking2.png"), (110, 60))
        ]
        self.jumping_surface = pygame.transform.scale(pygame.image.load("assets/DinoJumping.png"), (80, 100))

        self.x = x_pos
        self.y = y_pos

        self.current_image = 0
        self.image = self.running_sprites[self.current_image]
        self.rect = self.image.get_rect(center=(self.x, self.y))

        self.velocity = 15
        self.gravity = 1.5
        self.ducking = False
        self.is_jumping = False

    def update(self):
        self.animate()
        self.apply_gravity()
        self.move()

    def animate(self):
        self.current_image += 0.05
        if self.current_image >= 2:
            self.current_image = 0

        if self.ducking:
            self.image = self.ducking_sprites[int(self.current_image)]
        elif self.is_jumping:
            self.image = self.jumping_surface
        else:
            self.image = self.running_sprites[int(self.current_image)]

    def duck(self):
        self.ducking = True
        self.rect.centery = 460 

    def unduck(self):
        self.ducking = False
        self.rect.centery = 440

    def apply_gravity(self):
        if self.rect.centery <= 440:
            self.rect.centery += self.gravity

    def jump(self):
        if self.rect.centery >= 440:
            self.velocity = -30
            self.is_jumping = True

    def move(self):
        if self.is_jumping:
            self.rect.centery += self.velocity
            self.velocity += self.gravity
            if self.rect.centery >= 440:
                self.rect.centery = 440
                self.velocity = 0
                self.is_jumping = False



PAUSE_COLOR = (100, 100, 100, 200)
PAUSE_FONT_COLOR = (255, 255, 255)
paused = False

def draw_pause_menu():
    # Erstellen einer leicht transparenten dunkelgrauen Schicht
    pause_surface = pygame.Surface(SCREEN.get_size(), pygame.SRCALPHA)
    pause_surface.fill(PAUSE_COLOR)
    SCREEN.blit(pause_surface, (0, 0))

    # Zeichnen des Hintergrunds und des Spielers (optional)
    SCREEN.blit(ground, (ground_x, 0))
    SCREEN.blit(ground, (ground_x + 1280, 0))
    player_group.draw(SCREEN)

    pause_text = GAME_FONT.render("PAUSE", True, PAUSE_FONT_COLOR)
    pause_rect = pause_text.get_rect(center=(SCREEN.get_width() // 2, SCREEN.get_height() // 2 - 40))
    SCREEN.blit(pause_text, pause_rect)

    pygame.display.update()




# Hintergrund und Spielergruppe
ground = pygame.image.load("assets/background.png")
ground = pygame.transform.scale(ground, (1280, 900))
ground_x = 0
ground_rect = ground.get_rect(center=(640, 400))

player_group = pygame.sprite.GroupSingle()
player = Player(X_POSITION, Y_POSITION)
player_group.add(player)

# Hauptspiel-Schleife
while True:
    keys = pygame.key.get_pressed()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                paused = not paused  # Wechseln Sie den Pause-Zustand
            if event.key == pygame.K_SPACE or event.key == pygame.K_UP:
                player.jump()

    if not paused:
        if keys[pygame.K_DOWN]:
            player.duck()
        else:
            if player.ducking:
                player.unduck()

        player.update()

        GAME_SPEED += 0.0025
        ground_x -= 1

        SCREEN.blit(ground, (ground_x, 0))
        SCREEN.blit(ground, (ground_x + 1280, 0))
        player_group.draw(SCREEN)

        if ground_x <= -1280:
            ground_x = 0

    if paused:
        draw_pause_menu()

    pygame.display.update()
    CLOCK.tick(120)



SystemExit: 